In [1]:
!pip install -q transformers datasets accelerate tokenizers seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   

In [2]:
import datasets
import numpy as np
from transformers import BertTokenizerFast , DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer

## conll2003
- Conference on Computational Natural Language Learning
- It contains 4 entity types of NER as:
  - Person
  - Location
  - Organization
  - Miscellaneous

In [3]:
conll2003 = datasets.load_dataset("conll2003")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
conll2003["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [6]:
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
conll2003["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [8]:
conll2003["train"][1]["tokens"]

['Peter', 'Blackburn']

In [9]:
conll2003["train"][2]["tokens"]

['BRUSSELS', '1996-08-22']

In [10]:
conll2003["train"][3]["tokens"]

['The',
 'European',
 'Commission',
 'said',
 'on',
 'Thursday',
 'it',
 'disagreed',
 'with',
 'German',
 'advice',
 'to',
 'consumers',
 'to',
 'shun',
 'British',
 'lamb',
 'until',
 'scientists',
 'determine',
 'whether',
 'mad',
 'cow',
 'disease',
 'can',
 'be',
 'transmitted',
 'to',
 'sheep',
 '.']

In [11]:
conll2003["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [12]:
conll2003["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

# Observation
- ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

- [3, 0, 7, 0, 0, 0, 7, 0, 0]

- ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

> From above, we can say that the "EU" has given third place,which is B-Organization, "German" has given 7th position, which is representing it as B-Miscellaneous and so on.

### Fine-tuning is to get the specific info. from the data, or adding some info. to the results of the data

#### Model is already trained on internet based data, the trained model is called pretrained model.Here, we are performing **inferencing(prediction)** after training the pretrained model, now we have to retrain this pretrained model on specific data for NER task, which is called as finetuning.

In [13]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",num_labels=9) # uncased means not a case sensitive model

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
from transformers import pipeline
ner_pipeline = pipeline("ner",model=model,tokenizer=tokenizer)

Device set to use cuda:0


In [16]:
sent = "Sunny is data scientist and Generative AI Engineer"

In [17]:
ner_pipeline(sent)

[{'entity': 'LABEL_8',
  'score': 0.16461466,
  'index': 1,
  'word': 'sunny',
  'start': 0,
  'end': 5},
 {'entity': 'LABEL_6',
  'score': 0.14116232,
  'index': 2,
  'word': 'is',
  'start': 6,
  'end': 8},
 {'entity': 'LABEL_8',
  'score': 0.19885118,
  'index': 3,
  'word': 'data',
  'start': 9,
  'end': 13},
 {'entity': 'LABEL_4',
  'score': 0.14028883,
  'index': 4,
  'word': 'scientist',
  'start': 14,
  'end': 23},
 {'entity': 'LABEL_3',
  'score': 0.17429958,
  'index': 5,
  'word': 'and',
  'start': 24,
  'end': 27},
 {'entity': 'LABEL_8',
  'score': 0.17890927,
  'index': 6,
  'word': 'genera',
  'start': 28,
  'end': 34},
 {'entity': 'LABEL_0',
  'score': 0.15020978,
  'index': 7,
  'word': '##tive',
  'start': 34,
  'end': 38},
 {'entity': 'LABEL_8',
  'score': 0.19214778,
  'index': 8,
  'word': 'ai',
  'start': 39,
  'end': 41},
 {'entity': 'LABEL_0',
  'score': 0.19512562,
  'index': 9,
  'word': 'engineer',
  'start': 42,
  'end': 50}]

In [18]:
sent2 = "apple launch mobile while eating an apple which tastes like oranges."

In [19]:
ner_pipeline(sent2)

[{'entity': 'LABEL_6',
  'score': 0.18382423,
  'index': 1,
  'word': 'apple',
  'start': 0,
  'end': 5},
 {'entity': 'LABEL_6',
  'score': 0.14902963,
  'index': 2,
  'word': 'launch',
  'start': 6,
  'end': 12},
 {'entity': 'LABEL_6',
  'score': 0.18270165,
  'index': 3,
  'word': 'mobile',
  'start': 13,
  'end': 19},
 {'entity': 'LABEL_0',
  'score': 0.15798213,
  'index': 4,
  'word': 'while',
  'start': 20,
  'end': 25},
 {'entity': 'LABEL_6',
  'score': 0.14664389,
  'index': 5,
  'word': 'eating',
  'start': 26,
  'end': 32},
 {'entity': 'LABEL_4',
  'score': 0.14270914,
  'index': 6,
  'word': 'an',
  'start': 33,
  'end': 35},
 {'entity': 'LABEL_0',
  'score': 0.15566842,
  'index': 7,
  'word': 'apple',
  'start': 36,
  'end': 41},
 {'entity': 'LABEL_6',
  'score': 0.14927594,
  'index': 8,
  'word': 'which',
  'start': 42,
  'end': 47},
 {'entity': 'LABEL_6',
  'score': 0.16536099,
  'index': 9,
  'word': 'tastes',
  'start': 48,
  'end': 54},
 {'entity': 'LABEL_4',
  'scor

# Retraining

In [20]:
# Mapping of the data
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [21]:
# We don't require pos_tags and chunk_tags, to remove them, we are going to define one function
def tokenize_and_align_labels(examples, label_all_tokens=True):

    #tokeinze ids
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []


    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.

        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)

            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
q = tokenize_and_align_labels(conll2003["train"][0:1])

In [23]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
eu______________________________________ 3
rejects_________________________________ 0
german__________________________________ 7
call____________________________________ 0
to______________________________________ 0
boycott_________________________________ 0
british_________________________________ 7
lamb____________________________________ 0
._______________________________________ 0
[SEP]___________________________________ -100


In [24]:
q

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]]}

# Tokenization

In [25]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [26]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [27]:
sequence1 = "Using a Transformer network perform prediction"

In [28]:
tokenizer(sequence1)

{'input_ids': [101, 2478, 1037, 10938, 2121, 2897, 4685, 17547, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
token = tokenizer.tokenize(sequence1)
print(token)

['using', 'a', 'transform', '##er', 'network', 'perform', 'prediction']


In [30]:
ids = tokenizer.convert_tokens_to_ids(token)
print(ids)

[2478, 1037, 10938, 2121, 2897, 4685, 17547]


In [31]:
tokenizer.decode(ids)

'using a transformer network perform prediction'

## Hugging Face Working mechanism
> **Raw_text --> Tokens --> Token IDS --> Model Input(Embeddings,Attention Mask),etc.**

> **Raw_text --> Tokens --> Token Ids --> Embeddings --> Positional Encoding --> Self attention**

In [32]:
# for params in model.parameters():
#   print(params)

In [33]:
from transformers import TrainingArguments, Trainer

In [34]:
import evaluate

In [35]:
metric = evaluate.load("seqeval")

In [36]:
example = conll2003["train"][0]

In [37]:
label_list = conll2003["train"].features["ner_tags"].feature.names

In [38]:
for nertag in example["ner_tags"]:
  print(nertag)

3
0
7
0
0
0
7
0
0


In [39]:
labels = [label_list[i] for i in example["ner_tags"]]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [40]:
label=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [41]:
input=['EU','rejects','German','call','to','boycott','British','lamb','.']

actual_label=['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

pred_label=['B-ORG', 'O', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'] # "(Bert Model)"

In [42]:
metric.compute(predictions=[pred_label], references=[actual_label])

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'ORG': {'precision': 0.5,
  'recall': 1.0,
  'f1': 0.6666666666666666,
  'number': 1},
 'PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'overall_precision': 0.2,
 'overall_recall': 0.3333333333333333,
 'overall_f1': 0.25,
 'overall_accuracy': 0.2222222222222222}

In [43]:
# Mapping each token with each label as above we have done with the help of zip file
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [44]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }

In [45]:
#these are hyperparameter
args=TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none"  # Disable wandb logging
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

In [47]:
trainer=Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

<ipython-input-47-d09e5344868f>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.222000,0.068688,0.906608,0.920908,0.913702,0.980619


TrainOutput(global_step=878, training_loss=0.16318160906468002, metrics={'train_runtime': 163.1783, 'train_samples_per_second': 86.047, 'train_steps_per_second': 5.381, 'total_flos': 341387954498718.0, 'train_loss': 0.16318160906468002, 'epoch': 1.0})

In [49]:
model.save_pretrained("ner_model")

In [50]:
import shutil

# Model folder ka naam jo tu save kar chuka hai
model_folder = "/content/ner_model"

# Model ko zip me convert karo
shutil.make_archive("bert_model", 'zip', model_folder)

print("Model successfully zipped as bert_model.zip")

Model successfully zipped as bert_model.zip


In [51]:
from google.colab import files

# ZIP file ko download karne ke liye
files.download("bert_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
import zipfile

# ZIP file ka path
zip_path = "/content/bert_model.zip"

# Extract karne ke liye
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("./extracted_model")

print("Model successfully extracted!")

Model successfully extracted!


In [53]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [54]:
import json
config=json.load(open("/content/ner_model/config.json"))

In [55]:
config

{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'LABEL_0',
  '1': 'LABEL_1',
  '2': 'LABEL_2',
  '3': 'LABEL_3',
  '4': 'LABEL_4',
  '5': 'LABEL_5',
  '6': 'LABEL_6',
  '7': 'LABEL_7',
  '8': 'LABEL_8'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'LABEL_0': 0,
  'LABEL_1': 1,
  'LABEL_2': 2,
  'LABEL_3': 3,
  'LABEL_4': 4,
  'LABEL_5': 5,
  'LABEL_6': 6,
  'LABEL_7': 7,
  'LABEL_8': 8},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.48.3',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [56]:
conll2003["train"].features["ner_tags"].feature. names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [57]:
id2label = {str(i): label for i,label in enumerate(label_list)}

In [58]:
label2id = {label: str(i) for i,label in enumerate(label_list)}

In [59]:
config["id2label"]=id2label

In [60]:
config["label2id"]= label2id

In [61]:
json.dump(config,open("/content/ner_model/config.json","w"))

In [62]:
config=json.load(open("/content/ner_model/config.json"))

In [63]:
config

{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'O',
  '1': 'B-PER',
  '2': 'I-PER',
  '3': 'B-ORG',
  '4': 'I-ORG',
  '5': 'B-LOC',
  '6': 'I-LOC',
  '7': 'B-MISC',
  '8': 'I-MISC'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'O': '0',
  'B-PER': '1',
  'I-PER': '2',
  'B-ORG': '3',
  'I-ORG': '4',
  'B-LOC': '5',
  'I-LOC': '6',
  'B-MISC': '7',
  'I-MISC': '8'},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.48.3',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [64]:
from transformers import pipeline

In [65]:
model_name="/content/ner_model"

In [66]:
tokenizer = BertTokenizerFast.from_pretrained("/content/tokenizer")

In [67]:
model_fine_tuned=AutoModelForTokenClassification.from_pretrained("/content/ner_model")

In [68]:
model_fine_tuned=AutoModelForTokenClassification.from_pretrained(model_name)

In [69]:
nlp_pipeline=pipeline("ner",model=model_fine_tuned,tokenizer=tokenizer)

Device set to use cuda:0


In [70]:
example="Sunny is Data Scientist and Generative AI Engineer"

In [71]:
nlp_pipeline(example)

[{'entity': 'B-PER',
  'score': 0.9833086,
  'index': 1,
  'word': 'sunny',
  'start': 0,
  'end': 5}]

In [72]:
example = "my name is yashswi gandhak"

In [73]:
nlp_pipeline(example)

[{'entity': 'B-PER',
  'score': 0.97536975,
  'index': 4,
  'word': 'ya',
  'start': 11,
  'end': 13},
 {'entity': 'B-PER',
  'score': 0.9782361,
  'index': 5,
  'word': '##sh',
  'start': 13,
  'end': 15},
 {'entity': 'B-PER',
  'score': 0.9793367,
  'index': 6,
  'word': '##sw',
  'start': 15,
  'end': 17},
 {'entity': 'B-PER',
  'score': 0.97996235,
  'index': 7,
  'word': '##i',
  'start': 17,
  'end': 18},
 {'entity': 'I-PER',
  'score': 0.9936346,
  'index': 8,
  'word': 'gan',
  'start': 19,
  'end': 22},
 {'entity': 'I-PER',
  'score': 0.9921852,
  'index': 9,
  'word': '##dha',
  'start': 22,
  'end': 25},
 {'entity': 'I-PER',
  'score': 0.9925615,
  'index': 10,
  'word': '##k',
  'start': 25,
  'end': 26}]

In [74]:
!pip install huggingface_hub

In [75]:
from huggingface_hub import notebook_login

notebook_login()

In [76]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from huggingface_hub import HfApi

In [77]:
#Model directory (jisme fine-tuned model hai)
model_dir = "/content/ner_model"

In [79]:
# Push model to Hugging Face
model_fine_tuned.push_to_hub("NER-Model-Fine-Tuned")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Yashswi77/NER-Model-Fine-Tuned/commit/2453d9a7c8155c7e1a674c9bb5b7289f6d1127f4', commit_message='Upload BertForTokenClassification', commit_description='', oid='2453d9a7c8155c7e1a674c9bb5b7289f6d1127f4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Yashswi77/NER-Model-Fine-Tuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Yashswi77/NER-Model-Fine-Tuned'), pr_revision=None, pr_num=None)

In [80]:
# tokenizer.push_to_hub("your_username/your_model_name")
tokenizer.push_to_hub("Yashswi77/NER-Tokenizer")
print("✅ Model pushed successfully!")

✅ Model pushed successfully!
